Exercise 01:

In [ ]:
from sklearn.datasets import load_digits
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn import model_selection

digits = load_digits()
data = digits["data"]
images = digits["images"]
target = digits["target"]
target_names = digits["target_names"]

print(np.shape(images[0]))
#The images are of size 8x8

#Interpolate using nearest neighbor
img_nearest = images[3]
assert 2 == len (img_nearest.shape)
plt.figure()
plt.gray()
plt.imshow(img_nearest,interpolation = "nearest" ) # also try interpolation =" bicubic "
plt.show()

#Interpolate using bicubic
img_bicubic = images[3]
assert 2 == len (img_bicubic.shape)
plt.figure()
plt.gray()
plt.imshow(img_bicubic,interpolation = "bicubic" ) # also try interpolation =" bicubic "
plt.show()

x_3and9 = []
y_3and9 = []
image_3and9 = []
for i in range(len(data)):
    if target[i] == 3 or target[i] == 9:
        x_3and9.append(data[i])
        y_3and9.append(target[i]) 
        image_3and9.append(images[i])

X_train, X_test, y_train, y_test =\
    model_selection.train_test_split(x_3and9, y_3and9, test_size = 0.4, random_state = 0)


Exercise 04:

In [ ]:
def relu(t):
    if( t <= 0):
        return 0
    elif(t > 0):
        return t

# Calculate the loss for the svm approach
def loss(training_features, training_labels, beta, b, lam):
    n = len(training_labels)
    sum = 0
    for i in range(n):
        sum += relu(1-training_labels[i]*(training_features[i]*beta + b))
    return 1/2*np.transpose(beta)*beta + lam/n * sum

# Calculate the loss derivation by deriving for beta
def loss_derive_beta(training_features, training_labels, beta, b, lam):
    sum = 0
    n = len(training_labels)
    for i in range(n):
        if((training_labels[i] * beta + b) < 1):
            sum += (-training_labels[i] * np.transpose(training_features[i]))
    return beta + ((lam/n)*sum)

# Calculate the loss derivation by deriving for b
def loss_derive_b(training_labels, beta, b, lam):
    sum = 0
    n = len(training_labels)
    for i in range(n):
        if((training_labels[i] * beta + b) < 1):
            sum += (-training_labels[i])
    return (lam/n)*sum

def fit_svm(training_features, training_labels, lam):
    # Initialize required variables
    betas = []
    bs = []
    loss = []
    training_error = []
    # set starting values for gradient descent and learning rate
    betas.append(np.random.normal())
    bs.append(0)
    learning_rate = 0.1
    learning_rate_adjusted = False

    i = 0
    # start gradient descent
    while(True):
        # if the descent has at least 2 steps check if the training error is stagnating or further improving
        if(len(training_error) > 1):
            # if the training error is stagnating adjust the learning rate once
            # if it was alreade adjusted the process is finished
            if(math.isclose(training_error[i], training_error[i-1])):
                if(learning_rate_adjusted):
                   break
                else:
                    learning_rate = 1/learning_rate
                    learning_rate_adjusted = True

        loss.append(loss(training_features, training_labels, betas[i], bs[i], lam))
        training_error.append(np.average(loss))
        betas.append(betas[i] - learning_rate * loss_derive_beta(training_features, training_labels, betas[i], bs[i], lam))
        bs.append(bs[i] - learning_rate * loss_derive_b(training_labels, betas[i], bs[i], lam))
        i += 1

    return betas[i], bs[i]

def predict_svm(beta, b, test_features):
    predicted_labels = []
    for i in range(len(test_features)):
        predicted_labels.append(np.sign(test_features[i]*beta + b))
    return predicted_labels

# x_train is an N x D matrix, where N is the number of training instances and D is the number of relevant pixels
# y_train is a N-Dimensional vector containing the correct labels.
lam = 1
beta, b = fit_svm(X_train, y_train, lam)
predicted_labels = predict_svm(beta, b, X_test)


